<div>
    <div style="float:left;">
        <img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="50%" />
    </div>
    <div style="float:left;">
        <img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="50%"/>
    </div>
</div>

# JsMVA: Input variable plots
<hr style="border-top-width: 4px; border-top-color: #34609b;">

<!--<script src="JsRoot/scripts/JSRootCore.js?jq2d&onload=JsRootLoadedCall" type="text/javascript"></script>-->

In [ ]:
import ROOT
from ROOT import TFile, TMVA, TCut

## Enable JS visualization

In [ ]:
%jsmva on

## Declarations, building training and testing trees 

<b>For more details please see <a href="ROOTbooks-TMVA-JsMVA-UserInterface.ipynb">this</a> notebook.</b>

In [ ]:
outputFile = TFile( "TMVA.root", 'RECREATE' )

TMVA.Tools.Instance()

factory = TMVA.Factory(JobName="TMVAClassification", TargetFile=outputFile,
            V=True, Color=True, DrawProgressBar=True, Transformations=["I", "D", "P", "G","D"],
                       AnalysisType="Classification")

dataset = "tmva_class_example"
loader  = TMVA.DataLoader(dataset)

loader.AddVariable( "myvar1 := var1+var2", 'F' )
loader.AddVariable( "myvar2 := var1-var2", "Expression 2", 'F' )
loader.AddVariable( "var3",                "Variable 3", 'F' )
loader.AddVariable( "var4",                "Variable 4", 'F' )

loader.AddSpectator( "spec1:=var1*2",  "Spectator 1",  'F' )
loader.AddSpectator( "spec2:=var1*3",  "Spectator 2",  'F' )

if ROOT.gSystem.AccessPathName( "./tmva_class_example.root" ) != 0: 
    ROOT.gSystem.Exec( "wget https://root.cern.ch/files/tmva_class_example.root")
    
input = TFile.Open( "./tmva_class_example.root" )

# Get the signal and background trees for training
signal      = input.Get( "TreeS" )
background  = input.Get( "TreeB" )
    
# Global event weights (see below for setting event-wise weights)
signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(signal, signalWeight)
loader.AddBackgroundTree(background, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = signal
loader.fTreeB = background

loader.PrepareTrainingAndTestTree(SigCut=mycuts, BkgCut=mycutb,
            nTrain_Signal=0, nTrain_Background=0, SplitMode="Random", NormMode="NumEvents", V=False)

## Visualizing input variables

If we want to see a histogram of one of the variables readed from root file and passed to DataLoader, we have to invoke DataLoader.DrawInputVariable method. This method is available just in notebook, it's inseeted to DataLoader by using %jsmva on magic. The arguments of this function:

<table>
   <tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Description</th></tr>
   <tr>
       <td>variableName</td>
       <td>yes, 1.</td>
       <td>-</td>
       <td>The name of variable we want to visualize</td>
   </tr>
      <tr>
       <td>numBin</td>
       <td>yes, 2.</td>
       <td>100</td>
       <td>Number of bins used for creating the histogram</td>
   </tr>
      <tr>
       <td>processTrfs</td>
       <td>yes, 3.</td>
       <td>""</td>
       <td>List of transformations on input variables. Possible transformations: I (identity), N (normalize), D (decorrelation), P (PCA), U (uniform), G (Gaussian) </td>
   </tr>
</table>

In [ ]:
loader.DrawInputVariable("myvar1")

In [ ]:
loader.DrawInputVariable("myvar1", processTrfs=["G", "D"])

## Correlation matrix of input variables

To visualize the correlation between input variables we can draw a 2D histogram of correlation matrix. This can be done by using DataLoader.DrawCorrelationMatrix function (inserted to DataLoader by %jsmva on):
<table>
   <tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Description</th></tr>
   <tr>
       <td>className</td>
       <td>yes, 1.</td>
       <td>-</td>
       <td>Selects in which tree we want to draw the correlation matrix (Signal/Background)</td>
   </tr>
</table>

In [ ]:
loader.DrawCorrelationMatrix("Signal")

## Close the factory's output file

In [ ]:
outputFile.Close()